1. Họ và tên: Nguyễn Duy Hiếu
2. Lớp học: INT3405_1
3. Bài tập lớn môn học máy cuối kỳ
4. Chủ đề: Classify Toxic Question

In [ ]:
import os
import time
import numpy as np 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, LSTM, GRU, CuDNNGRU


import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
import pandas as pd



**ĐỌC DỮ LIỆU VÀO**

In [ ]:
df_train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
print(df_train.shape)
df_train.head()

In [ ]:
df_test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
print(df_test.shape)
df_test.head()

**PHÂN TÍCH DỮ LIỆU**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
target = df_train['target']
target_1 = 0
for target_value in target:
    if target_value == 1:
        target_1 += 1
print("Số câu hỏi trong tệp train:", len(target))
print("Số câu hỏi được gán nhãn là 1:", target_1)
myLabels = ["insincere question", "sincere question"]
myCounts = [target_1, len(target) - target_1]
plt.pie(myCounts, labels = myLabels, autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')
plt.show()

In [ ]:
from wordcloud import WordCloud
print("Word cloud thể  hiện các câu hỏi sincere: ")
sincere_wordcloud = WordCloud(width=700, height=500, background_color='black', min_font_size=10).generate(str(df_train[df_train["target"] == 0]["question_text"]))
plt.figure(figsize=(10,9), facecolor=None)
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
from wordcloud import WordCloud
print("Word cloud thể  hiện các câu hỏi insincere: ")
sincere_wordcloud = WordCloud(width=700, height=500, background_color='black', min_font_size=10).generate(str(df_train[df_train["target"] == 1]["question_text"]))
plt.figure(figsize=(10,9), facecolor=None)
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

**TIỀN XỬ LÝ**

In [ ]:
df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=2018)

embed_size = 300 # độ dài mỗi vector từ
max_features = 50000 # số lượng các từ được sử dụng tối đa
maxlen = 100 # max number of words in a question to use

X_train = df_train["question_text"].fillna("_NA_").values
X_val = df_val["question_text"].fillna("_NA_").values
X_test = df_test["question_text"].fillna("_NA_").values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

y_train = df_train['target'].values
y_val = df_val['target'].values

**NAIVE BAYES**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

MulNB = MultinomialNB()
MulNB.fit(X_train,y_train)

In [ ]:
model = MulNB
predictions = model.predict(X_train)
train_acc = accuracy_score(y_train, predictions)
train_f1 = f1_score(y_train, predictions)*4
print(f"Training accuracy: {train_acc:.2%}, F1 score: {train_f1:.4f}") 
test_predictions = model.predict(X_val)
test_acc = accuracy_score(y_val, test_predictions) 
test_f1 = f1_score(y_val, test_predictions)*4
print(f"Testing accuracy:  {test_acc:.2%}, F1 score: {test_f1:.4f}")

**SỬ DỤNG LSTM**

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
clf = Model(inputs=inp, outputs=x)
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(clf.summary())

In [ ]:
clf.fit(X_train, y_train, batch_size=512, epochs=2, validation_data=(X_val, y_val))

In [ ]:
pred_val_y = clf.predict([X_val], batch_size=1024, verbose=1)
maxLSTM = 0
maxThresh = 0
threshMatrixLSTM = []
metricsF1MatrixLSTM = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(y_val, (pred_val_y>thresh).astype(int))))
    threshMatrixLSTM.append(thresh)
    metricsF1MatrixLSTM.append(metrics.f1_score(y_val, (pred_val_y>thresh).astype(int)))
    if (metrics.f1_score(y_val, (pred_val_y>thresh).astype(int)) > maxLSTM):
        maxLSTM = metrics.f1_score(y_val, (pred_val_y>thresh).astype(int))
        maxThresh = thresh
plt.plot(threshMatrixLSTM, metricsF1MatrixLSTM)
plt.xlabel('thresh')
plt.ylabel('F1_Score')

In [ ]:
maxLSTM

In [ ]:
maxThresh

In [ ]:
pred_test_y = clf.predict([X_test], batch_size=1024, verbose=1)

**Sử dụng GRU Model**

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
model.fit(X_train, y_train, batch_size=512, epochs=2, validation_data=(X_val, y_val))

In [ ]:
pred_noemb_y_val = model.predict([X_val], batch_size=1024, verbose=1)
threshMatrixGRU = []
metricsF1MatrixGRU = []
maxGRU = 0
maxThresh = 0
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(y_val, (pred_noemb_y_val>thresh).astype(int))))
    threshMatrixGRU.append(thresh)
    metricsF1MatrixGRU.append(metrics.f1_score(y_val, (pred_noemb_y_val>thresh).astype(int)))
    if (metrics.f1_score(y_val, (pred_noemb_y_val>thresh).astype(int)) > maxGRU): 
        maxGRU = metrics.f1_score(y_val, (pred_noemb_y_val>thresh).astype(int))
        maxThresh = thresh
plt.plot(threshMatrixGRU,metricsF1MatrixGRU)
plt.xlabel('thresh')
plt.ylabel('F1_score')

In [ ]:
maxGRU

In [ ]:
maxThresh

In [ ]:
pred_test_y_GRU = clf.predict([X_test], batch_size=1024, verbose=1)

**SUBMIT**

In [ ]:
df_submission = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')
df_submission['prediction'] = (0.34*pred_test_y).astype(int) # Best threshold for f1-Score
# df_submission.head()

In [ ]:
df_submission.to_csv('submission.csv',index=False)